### SQL row convert into column

In [2]:
%load_ext sql

In [3]:
%sql sqlite://

Connecting to 'sqlite://'

In [4]:
%%sql
create table replayment (
    c_id int,
    service str,
    s_date date);



Running query in 'sqlite://'

++
||
++
++

In [5]:
%%sql
insert into replayment(c_id,service,s_date) values 
(1,'A','2025-01-01'),(1,'B','2025-01-01'),(1,'C','2025-01-01'),(1,'A','2025-01-02'),
(2,'A','2025-01-01'),(2,'B','2025-01-01'),(2,'C','2025-01-01'),(3,'A','2025-01-01'),
(4,'A','2025-01-01'),(5,'A','2025-01-01'),(5,'C','2025-01-03'),(6,'A','2025-01-01');

Running query in 'sqlite://'

12 rows affected.

++
||
++
++

In [16]:
%%sql

with table1 as(
select c_id,service,s_date,count(*) as count1 from replayment group by c_id,service)

select * from table1;

Running query in 'sqlite://'

c_id,service,s_date,count1
1,A,2025-01-01,2
1,B,2025-01-01,1
1,C,2025-01-01,1
2,A,2025-01-01,1
2,B,2025-01-01,1
2,C,2025-01-01,1
3,A,2025-01-01,1
4,A,2025-01-01,1
5,A,2025-01-01,1
5,C,2025-01-03,1


Convert row into column

In [22]:

%%sql

with table1 as(
select c_id,service,s_date,count(*) as count1 from replayment group by c_id,service)
--Convert row into column
select c_id,s_date,
sum(case when service='A' then count1 else 0 end) as Count_A,
sum(case when service='B' then count1 else 0 end) as Count_B,
sum(case when service='C' then count1 else 0 end) as Count_C
from table1 group by c_id,s_date ;


Running query in 'sqlite://'

c_id,s_date,Count_A,Count_B,Count_C
1,2025-01-01,2,1,1
2,2025-01-01,1,1,1
3,2025-01-01,1,0,0
4,2025-01-01,1,0,0
5,2025-01-01,1,0,0
5,2025-01-03,0,0,1
6,2025-01-01,1,0,0


In [6]:
%%sql
with table1 as(
select c_id,service,s_date,count(*) as count1 from replayment group by c_id,service),
--Convert row into column
table2 as (
select c_id,s_date,count1,
sum(case when service='A' then count1 else 0 end) as Count_A,
sum(case when service='B' then count1 else 0 end) as Count_B,
sum(case when service='C' then count1 else 0 end) as Count_C
from table1 group by c_id,s_date )
    
select c_id,s_date,'A' as Service ,count1 as count_value from table2 where Count_A >0
union all
select c_id,s_date,'B' as Service ,count1 as count_value from table2 where Count_B >0
union all
select c_id,s_date,'C' as Service ,count1 as count_value from table2 where Count_C >0;

Running query in 'sqlite://'

c_id,s_date,Service,count_value
1,2025-01-01,A,2
2,2025-01-01,A,1
3,2025-01-01,A,1
4,2025-01-01,A,1
5,2025-01-01,A,1
6,2025-01-01,A,1
1,2025-01-01,B,2
2,2025-01-01,B,1
1,2025-01-01,C,2
2,2025-01-01,C,1
